In [16]:
import numpy as np
import matplotlib.pyplot as plt
from multiprocessing import Pool
from multiprocessing.dummy import Pool as ThreadPool
import time
%matplotlib inline
from snn1337.layers import *
from snn1337.spiking_from_lasagne import *

In [17]:
%run fully_connected_for_mnist.ipynb
%run for_snn_debug.ipynb

In [18]:
from mnist import load_dataset
X_train,y_train,X_val,y_val,X_test,y_test = load_dataset()

In [19]:
input_X = T.tensor4("X")
input_shape = [None,1,28,28]
target_y = T.vector("target Y integer",dtype='int32')

#### Создаём полносвязную сеть на lasagne, не обучеам её!!!

In [20]:
net = fully_connected(input_X, input_shape, 1, [16])

In [21]:
thr = [0.1, 0.2]
t_max = 100
num_epoch = 5
img_num = 1

outputs = []
weights = []
w_before = []
w_after = []

In [22]:
# попробуем подобрать размеры окна для stpd

In [23]:
%%time
# создаём из неё спайковую сеть, при этом сделаем тест на то что спайковая сеть учится с stdp
for epoch in range(num_epoch):
    for img_ind in range(img_num): 
        
        # создаём сеть из lasagne
        snn_net = spiking_from_lasagne(net, thr)
        
        # задаём ей веса
        if (len(weights) != 0):
            snn_net.set_all_params_values(weights[-1])
        
        # получаем ответ для данных    
        outputs.append(snn_net.get_output_for(X_train[img_ind], t_max))
        
        w_before = snn_net.get_all_params_values()
        
        # обучаем
        snn_net.learning()
        
        # сохраняем веса
        weights.append(snn_net.get_all_params_values())
        w_after = snn_net.get_all_params_values()

CPU times: user 9.49 s, sys: 40 ms, total: 9.53 s
Wall time: 9.46 s


In [24]:
print(w_before == w_after)

True


In [25]:
print(np.array(weights).shape)
print(weights[0] == weights[-1])

(5, 2)
True


### Видим, что веса спайковой сети не меняются